In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
import networkx as nx
from wordcloud import WordCloud # https://github.com/amueller/word_cloud


In [29]:
wd = '/projects/ps-yeolab3/bay001/tbos/input_norm/input_norm_latest/'
bedhead = ['chrom','start','end','l10p','l2fc','strand','annotation','gene']
gene2name = '/projects/ps-yeolab/genomes/hg19/ensembl_to_gene_name_biomart.txt'
clip2hta = '/home/bay001/projects/tbos_clipseq_20160809/permanent_data/clip2hta.txt'
hta2peak = pd.read_table(clip2hta,names=['hta','peak'])

clips = glob.glob(os.path.join(wd,"*.bed.annotated"))
#clips = clips[:1]

#clip = clips[0]
def gencode2ensembl(row):
    try:
        return row['gene'].split('.')[0]
    except Exception as e:
        return 'intergenic'
def getpeaks(clip, l10p = 3, l2fc = 3):
    df = pd.read_table(clip,names=bedhead)
    df = df[(df['l10p']>l10p) & (df['l2fc']>l2fc)]
    df['ensembl'] = df.apply(gencode2ensembl,axis=1)
    return df

def peak2rbp(clip,dfz):
    hta = ''
    try:
        hta = dfz[dfz['peak']==os.path.basename(clip).replace('.annotated','.trimmed.sorted.annotated')]['hta'].iloc[0]
    except IndexError as e:
        print('no known RBP for {}'.format(os.path.basename(clip)))
        return '-'
    return hta.replace(
        'hta_sepscore_yeo_aug15_all_','').replace(
        '_sepscore','').replace(
        '_over','').replace(
        '_kd','').replace(
        '_over1','')

def make_wordcloud(clip,outdir,l10p=3,l2fc=3,min_peak_count=0):
    clip_name = os.path.basename(clip)
    hta2peak = pd.read_table(clip2hta,names=['hta','peak'])
    dfx = pd.read_table(gene2name)
    # print(dfx.head())
    dfy = getpeaks(clip,l10p,l2fc)
    # print(dfy.head())
    num_peaks = dfy.shape[0]
    df = pd.merge(dfy,dfx,how='left',left_on='ensembl',right_on='Gene ID')
    df['Counts'] = df.groupby(['Associated Gene Name'])['Associated Gene Name'].transform('count')
    df.sort_values('Counts',ascending=False,inplace=True)
    df = df[df['Counts'] > min_peak_count]
    return df
    """
    wordcloud = WordCloud().generate(str(list(df['Associated Gene Name'])).replace(',','').replace('\'',''))
    plt.imshow(wordcloud)
    plt.title("{}, peaks: {}".format(peak2rbp(clip,hta2peak),num_peaks))
    plt.axis("off")
    plt.savefig(os.path.join(outdir,clip_name)+'.png')
    return df"""

In [31]:
out = '/projects/ps-yeolab3/bay001/tbos/images/wordclouds/'
i = 0
for clip in clips[:2]:
    df = make_wordcloud(clip,out)
    print(os.path.basename(clip),i),
    i = i + 1

('A9_01_01.basedon_A9_01_01.peaks.l2inputnormnew.bed.compressed.bed.annotated', 0) ('WTL_01_01.basedon_WTL_01_01.peaks.l2inputnormnew.bed.compressed.bed.annotated', 1)


In [33]:
clip

'/projects/ps-yeolab3/bay001/tbos/input_norm/input_norm_latest/WTL_01_01.basedon_WTL_01_01.peaks.l2inputnormnew.bed.compressed.bed.annotated'

In [44]:
df[df['Associated Gene Name'].str.contains('LUC')]

,chrom,start,end,l10p,l2fc,strand,annotation,gene,ensembl,Gene ID,Associated Gene Name,Counts
1497,chr17,48823337,48823364,13.108485,3.569893,+,CDS|contained|ENSG00000108848.11,ENSG00000108848.11,ENSG00000108848,ENSG00000108848,LUC7L3,5.0
1511,chr17,48818596,48818608,12.308282,3.133323,+,CDS|partial|ENSG00000108848.11,ENSG00000108848.11,ENSG00000108848,ENSG00000108848,LUC7L3,5.0
1447,chr17,48797095,48797194,69.255222,3.112583,+,CDS|partial|ENSG00000108848.11,ENSG00000108848.11,ENSG00000108848,ENSG00000108848,LUC7L3,5.0
1453,chr17,48797014,48797038,25.316647,3.470358,+,5utr|contained|ENSG00000108848.11,ENSG00000108848.11,ENSG00000108848,ENSG00000108848,LUC7L3,5.0
1456,chr17,48796976,48797014,21.326942,3.437621,+,5utr|contained|ENSG00000108848.11,ENSG00000108848.11,ENSG00000108848,ENSG00000108848,LUC7L3,5.0
9731,chr16,256003,256034,4.363984,3.014678,-,CDS|contained|ENSG00000007392.12,ENSG00000007392.12,ENSG00000007392,ENSG00000007392,LUC7L,1.0
11598,chr7,139045038,139045071,15.274832,3.091846,+,CDS|partial|ENSG00000146963.13,ENSG00000146963.13,ENSG00000146963,ENSG00000146963,C7orf55-LUC7L2,1.0


In [42]:
df

,chrom,start,end,l10p,l2fc,strand,annotation,gene,ensembl,Gene ID,Associated Gene Name,Counts
9362,chr16,2142139,2142182,18.186342,4.666755,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9312,chr16,2158625,2158695,26.977345,5.161520,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9450,chr16,2185597,2185621,12.704124,5.065304,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9304,chr16,2152909,2152971,30.740812,4.266217,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9305,chr16,2161118,2161182,30.505247,4.751644,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9306,chr16,2139284,2139334,28.775856,4.187145,-,3utr|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9307,chr16,2168119,2168187,28.642122,3.604213,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9308,chr16,2152592,2152634,28.177564,3.831814,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9309,chr16,2159035,2159103,28.019688,4.644729,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
9310,chr16,2161744,2161810,28.019688,4.644729,-,CDS|contained|ENSG00000008710.13,ENSG00000008710.13,ENSG00000008710,ENSG00000008710,PKD1,110.0
